In [1]:
import duckdb
import altair as alt
import polars as pl
import yfinance as yf

c_datetime_local = "datetime_local"
c_close = "close"
c_symbol = "symbol"
c_resolution = "resolution"
EXPORT_PATH = "../../assets/images/benner"

In [2]:
if True:
    resolution = "1wk"

    df_ohlc = pl.from_dataframe(yf.Ticker("^GSPC").history(period="max", interval=resolution).reset_index()).with_columns(
        pl.col("Date").alias(c_datetime_local),
        pl.col("Close").alias(c_close),
        pl.lit("^GSPC").alias(c_symbol),
        pl.lit(resolution).alias(c_resolution),
    )
    with pl.Config(tbl_formatting="MARKDOWN"):
        print(
            df_ohlc.group_by(pl.col(c_symbol, c_resolution)).agg(
                pl.col(c_datetime_local).min().alias("min"),
                pl.col(c_datetime_local).max().alias("max"),
                pl.col(c_datetime_local).count().alias("count"),
            )
        )
else:
    df_ohlc = pl.read_csv("SPPUKA.csv").with_columns(
        pl.col("observation_date").cast(pl.Date).alias(c_datetime_local),
        pl.col("SPPUKA").alias(c_close),
    )

df_ohlc

shape: (1, 5)
| symbol | resolution | min                            | max                            | count |
| ---    | ---        | ---                            | ---                            | ---   |
| str    | str        | datetime[ms, America/New_York] | datetime[ms, America/New_York] | u32   |
|--------|------------|--------------------------------|--------------------------------|-------|
| ^GSPC  | 1wk        | 1927-12-26 00:00:00 EST        | 2026-02-02 00:00:00 EST        | 5119  |


Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,datetime_local,close,symbol,resolution
"datetime[ms, America/New_York]",f64,f64,f64,f64,i64,f64,f64,"datetime[ms, America/New_York]",f64,str,str
1927-12-26 00:00:00 EST,17.66,17.66,17.66,17.66,0,0.0,0.0,1927-12-26 00:00:00 EST,17.66,"""^GSPC""","""1wk"""
1928-01-02 00:00:00 EST,17.76,17.76,17.549999,17.66,0,0.0,0.0,1928-01-02 00:00:00 EST,17.66,"""^GSPC""","""1wk"""
1928-01-09 00:00:00 EST,17.5,17.58,17.35,17.58,0,0.0,0.0,1928-01-09 00:00:00 EST,17.58,"""^GSPC""","""1wk"""
1928-01-16 00:00:00 EST,17.290001,17.48,17.26,17.48,0,0.0,0.0,1928-01-16 00:00:00 EST,17.48,"""^GSPC""","""1wk"""
1928-01-23 00:00:00 EST,17.639999,17.709999,17.52,17.690001,0,0.0,0.0,1928-01-23 00:00:00 EST,17.690001,"""^GSPC""","""1wk"""
…,…,…,…,…,…,…,…,…,…,…,…
2026-01-05 00:00:00 EST,6892.189941,6978.359863,6891.560059,6966.279785,26993190000,0.0,0.0,2026-01-05 00:00:00 EST,6966.279785,"""^GSPC""","""1wk"""
2026-01-12 00:00:00 EST,6944.120117,6986.330078,6885.740234,6940.009766,26112200000,0.0,0.0,2026-01-12 00:00:00 EST,6940.009766,"""^GSPC""","""1wk"""
2026-01-19 00:00:00 EST,6865.240234,6934.75,6789.049805,6915.609863,21784530000,0.0,0.0,2026-01-19 00:00:00 EST,6915.609863,"""^GSPC""","""1wk"""


In [3]:
print([0, 8, 8+9, 8+9+10])

relation = duckdb.sql(f"""
WITH staging AS (
    SELECT
{c_datetime_local},
{c_close},
-- ft_dt_benner_highs
(YEAR({c_datetime_local}) - 1729) % 27 high_cycle,
(YEAR({c_datetime_local}) - 1735) % 27 low_cycle,
(YEAR({c_datetime_local}) - 1911) % 54 panic_cycle,
LEAD({c_close}, 1, 0) OVER (ORDER BY {c_datetime_local}) - {c_close} AS forward_returns,
FROM df_ohlc
)

select
YEAR({c_datetime_local}),
-- MIN(high_cycle),
SUM(forward_returns),
from staging
-- where high_cycle in [0, 8, 17]
where high_cycle in [1, 9, 18]
-- where year({c_datetime_local}) in [2015, 2016]
group by year({c_datetime_local})
order by year({c_datetime_local})
""")

relation


[0, 8, 17, 27]


┌────────────────────────┬──────────────────────┐
│ "year"(datetime_local) │ sum(forward_returns) │
│         int64          │        double        │
├────────────────────────┼──────────────────────┤
│                   1927 │                  0.0 │
│                   1936 │   3.6999998092651367 │
│                   1946 │  -2.8899993896484375 │
│                   1954 │   10.400001525878906 │
│                   1963 │   11.389999389648438 │
│                   1973 │  -26.209999084472656 │
│                   1981 │   -13.92999267578125 │
│                   1990 │  -36.970001220703125 │
│                   2000 │   -143.1199951171875 │
│                   2008 │   -510.6700439453125 │
│                   2017 │        466.169921875 │
├────────────────────────┴──────────────────────┤
│ 11 rows                             2 columns │
└───────────────────────────────────────────────┘

In [6]:
relation = duckdb.sql(f"""
WITH staging AS (
    SELECT
{c_datetime_local},
{c_close},
(YEAR({c_datetime_local}) - 1729) % 27 high_cycle,
(YEAR({c_datetime_local}) - 1735) % 27 low_cycle,
(YEAR({c_datetime_local}) - 1911) % 54 panic_cycle,
LEAD({c_close}, 1, 0) OVER (ORDER BY {c_datetime_local}) - {c_close} AS forward_returns,
FROM df_ohlc
)

select
high_cycle,
MEAN(forward_returns / {c_close}) mean_change,
from staging
where high_cycle in [0, 8, 17]
or high_cycle in [1, 9, 18]
group by high_cycle
order by high_cycle
""")

with pl.Config(tbl_formatting="MARKDOWN", tbl_rows=20):
    print(relation.pl())


shape: (6, 2)
| high_cycle | mean_change |
| ---        | ---         |
| i64        | f64         |
|------------|-------------|
| 0          | -0.002518   |
| 1          | -0.002993   |
| 8          | 0.001094    |
| 9          | -0.000875   |
| 17         | 0.003714    |
| 18         | 0.002427    |


In [7]:
high_prices_highlights = alt.Chart(
    pl.DataFrame({
        'cycle_period': [0, 8, 8+9, 8+9+10],
    })
).mark_rule(color='red', strokeWidth=4, opacity=0.5).encode(
    x='cycle_period'
)

chart = duckdb.sql(f"""
WITH staging AS (
    SELECT
{c_datetime_local},
{c_close},
(YEAR({c_datetime_local}) - 1729) % 27 high_cycle,
(YEAR({c_datetime_local}) - 1735) % 27 low_cycle,
(YEAR({c_datetime_local}) - 1911) % 54 panic_cycle,
LEAD({c_close}, 1, 0) OVER (ORDER BY {c_datetime_local}) - {c_close} AS forward_returns,
FROM df_ohlc
)

select
    high_cycle,
    mean(forward_returns) mean_returns,
    mean(forward_returns / {c_close}) mean_change,
from staging
where high_cycle >= 0
group by high_cycle
order by high_cycle
""").pl().plot.bar(x="high_cycle", y="mean_change")

(chart + high_prices_highlights).save(EXPORT_PATH + "/high-prices.svg")
chart + high_prices_highlights


alt.LayerChart(...)